# Introduction to data science: classification

This is where the **science** comes in!

Data science depends hugely on data, but the 'science' part requires us to select the best (i.e. optimal) model -- and this means having an objective measure of 'best' and a way to prove that you have found it.

First we'll import some data. I'm using an extract from the Rock Property Catalog, https://subsurfwiki.org/wiki/Rock_Property_Catalog

## Interlude: what is NumPy?

- Motivation
- Making arrays
- Methods on arrays
- Accessing rows and columns
- Broadcasting
- Modules

If you're writing a loop, you're doing it wrong!

## Back to the data

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(*X.T)

In [ ]:
import matplotlib.pyplot as plt

LITHS = ['limestone', 'dolomite', 'shale']

def lith_index(y):
    return [LITHS.index(lith) for lith in y]

plt.scatter(*X.T, c=lith_index(y))

## A linear model: SVM


The **support vector machine** is a reliable classifier.

In [ ]:
from sklearn.svm import SVC



In [ ]:
plt.scatter(*X.T, c=lith_index(y), s=80, cmap='coolwarm')
plt.scatter(*X.T, c=lith_index(y_pred), cmap='bwr')

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y, y_pred)

**❓ What do we think? Are we satisfied?**

<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;

## Scoring

Scores matter in all machine learning tasks. It is very common to see people reporting only **accuracy** for classification tasks, or only **R<sup>2</sup>** for regression tasks. It is almost never enough to only look at (or report) the 'obvious' score -- especially for multiclass problems like this one, and especially when there is class imbalance.

**This is one of the most important review papers in all of machine learning: [Raschka 2018](https://arxiv.org/abs/1811.12808). Read it and share it.**

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y, y_pred))

## Validation

We should not train the model then check its accuracy only on that same training dataset. It's cheating.

Let's hold out some validation data, or 'blind' data.

In [ ]:
from sklearn.model_selection import train_test_split

train_test_split?

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

### EXERCISE

Now let's train a model _on only the training data_ and validate it properly _on only the test data_.

**❓ Do we think the score will be better or worse than before?**

We need to think about:

- Independence: can you shuffle the data wthout losing information?
- Identical distributions: are all the data from the same distribution?
- Reproducibility: what can we do to make this reproducible?
- Stratification: How can we deal with class imbalance?

Can you demonstrate that the test sample is fair and reproducible?

## A non-linear SVM model

If we employ the **kernel trick** we can fit a nonlinear model. Scikit-learn's `SVC` actually uses this by default:

In [ ]:
svc = SVC()  # Default is kernel='rbf'

svc.fit(X_train, y_train)

y_pred = svc.predict(X_test)

plt.scatter(*X_test.T, c=lith_index(y_test), s=80, cmap='coolwarm')
plt.scatter(*X_test.T, c=lith_index(y_pred), cmap='bwr')

The score is muuuuuuuuuuuch worse, especially for the minority class:

In [ ]:
print(classification_report(y_test, y_pred))

**❓ What could the problem be?**

<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;

### Scale matters

We are not looking at the data in its 'true' numerically isotropic space:

In [ ]:
plt.scatter(*X_train.T)

## Standardize the data

It's essential to train SVMs on scaled data, usually the Z-scores of your data, i.e. zero mean, unit variance. This ensures that the different scales of the features is not causing a problem.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X_train)

X_train_sc = scaler.transform(X_train)
X_test_sc = scaler.transform(X_test)

This doesn't change how the data are **relatively** distributed:

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(15, 5))

axs[0].scatter(*X_train.T)
axs[1].scatter(*X_train_sc.T)

But it dramatically changes how they are disitributed in absolute terms:

In [ ]:
import seaborn as sns

fig, axs = plt.subplots(ncols=2, figsize=(15, 5))
sns.kdeplot(X_train, ax=axs[0])
sns.kdeplot(X_train_sc, ax=axs[1])

### EXERCISE

Re-fit the **linear** model and look at the scores.

Solving one problem gives us a new one. Now we have a new pitfall: it is essential to scale the data now before inference -- although the model will happily make (terrible) predictions.

This is known as an "out of distribution" or OOD error, and it's a classic pitfall in machine learning.

## Put everything in a pipeline

This is the most flexible way to compose data pipelines in `sklearn`. It is usually better than implementing everything individually in a stepwise manner.

For now, it won't change anything.

In [ ]:
from sklearn.pipeline import make_pipeline

pipe = 

And now the nonlinear SVM is basically identical to the linear one:

## Hyperparameter tuning

Most algorithms have **hyperparameters** which control how the algorithm learns -- and which often implement some kind of **regularization**. This means different things for different algorithms, but in general it involves simplifying or smoothing the model in some way.

For the [support vector machine](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html), there is a parameter `C` to control the model complexity (high `C` means high complexity and low regularization).

### EXERCISE

Write a loop to step over values of the regularization parameter `C`. Start with:

    C = np.logspace(-3, 5, 17)
    train, test = [], []
    for Ci in C:
        # Instantiate the pipeline, making you use `SVC(C=Ci)`.
        # Get the f1_score for both train and test sets.
        # Gather these scores in two lists.

When you have 2 lists of mean scores, plot them and compare how they vary with C. Put the x-axis on a log scale with `plt.xscale('log')`

In [ ]:
from sklearn.metrics import f1_score

C = np.logspace(-3, 5, 17)

train, test = [], []
for Ci in C:
    pipe =   # YOUR CODE HERE


plt.plot(C, test)
plt.plot(C, train, c='r')
plt.xscale('log')
plt.ylim(1.0, 0.65)

**❓ Do you see a potential issue here?**

<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;<br />&nbsp;

Sometimes you will get a lucky test split, sometimes not. This feels a bit aribtrary. And we might overfit this model selection process to the particular test set that we have.

## Cross validation

Instead of choosing a single holdout set for evaluation, we can choose many. This is useful for tuning hyperparameters, for example.

But then we have more ways to leak information into our evaluation, so it is smart to continue to hold out our `test` set. **In general, you should always have at least 3 datasets: `train`, `val` (for tuning) and `test` (for the final test).

Fair evaluation is hard!

In [ ]:
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import make_scorer

pipe = make_pipeline(StandardScaler(), SVC(C=0.1))
scoring = make_scorer(f1_score, average='weighted')
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cross_validate(pipe,
               X_train, y_train,  # Will be split into train and val sets.
               scoring=scoring,
               cv=skf,
               return_train_score=True,
               # Also useful: groups
              )

Notice that we no longer use `test` during the model fitting process.

### EXERCISE

Use cross-validation in the loop over values of `C`. Start with:

    C = np.logspace(-3, 5, 17)
    mean_val, mean_train = [], []
    for Ci in C:
        # Instantiate the pipeline, making you set `C=Ci`.
        # Do the cross validation model fits with `cross_validate()`.
        # Gather the means of the test and train scores.

Then make the plot, as before.

In [ ]:
mean_val, mean_train = [], []

for Ci in C:
    # YOUR CODE HERE
    
plt.plot(C, mean_val)
plt.plot(C, mean_train, c='r')
plt.xscale('log')
plt.ylim(1, 0.75)

## Explore the model zoo!

### EXERCISE

- Add `Vs` (shear velocity) to the features and see if it improves the prediction quality.
- Choose another algorithm to try a prediction with, and implement it in a pipeline.
- Choose a hyperparameter of the new algorithm and tune it. (If you have done this kind of thing before, try tuning 2 or 3 hyperparameters with grid or random search.)

## Test

When you have tuned the predictor and are satisfied that it is as good as it can be, you can test your prediction power.

In [ ]:
y_pred = pipe.predict(X_test)

print(classification_report(y_test, y_pred))

If you are satisfied (think hard about what this means... you really have to decide before you start the model fitting process) then you are ready to fit the final model. If not, you must start all over again.

## Using this model

We do not want to use this model &mdash; if we like its performance then we should now retrain it on all the data. Presumably, this new model will be at least as good as the one trained on the training set, we just don't have a way to check it now 😬

In [ ]:
C_opt = C[np.argmax(mean_val)]
C_opt

In [ ]:
scaler = StandardScaler().fit(X)
X_ = scaler.transform(X)
svc = SVC(C=10**3.5).fit(X_, y)

There is no way for us to test this model, but we should monitor it in production.

---
&copy; 2024 Matt Hall / Equinor CC BY